In [4]:
!pip install albumentations

  Using cached numpy-1.24.4-cp38-cp38-win_amd64.whl.metadata (5.6 kB)
  Using cached typing_extensions-4.11.0-py3-none-any.whl.metadata (3.0 kB)
Using cached numpy-1.24.4-cp38-cp38-win_amd64.whl (14.9 MB)
Using cached typing_extensions-4.11.0-py3-none-any.whl (34 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
object-detection 0.1 requires apache-beam, which is not installed.
object-detection 0.1 requires avro-python3, which is not installed.
object-detection 0.1 requires contextlib2, which is not installed.
object-detection 0.1 requires Cython, which is not installed.
object-detection 0.1 requires lvis, which is not installed.
object-detection 0.1 requires pandas, which is not installed.
object-detection 0.1 requires pycocotools, which is not installed.
sacrebleu 2.2.0 requires portalocker, which is not installed.
sacrebleu 2.2.0 requires regex, which is not installed.
sacrebleu 2.2.0 requires tabulate>=0.8.9, which is not installed.
tf-models-official 2.16.0 requires Cython, which is not installed.
tf-models-official 2.16.0 requires gin-config, which is not installed.
tf-models-off

In [5]:
!pip install torch

In [3]:
import os
import xml.etree.ElementTree as ET
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [ ]:
# Function to parse XML files and extract bounding boxes
def parse_voc_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    boxes = []
    for member in root.findall("object"):
        xmin = int(member.find("bndbox").find("xmin").text)
        ymin = int(member.find("bndbox").find("ymin").text)
        xmax = int(member.find("bndbox").find("xmax").text)
        ymax = int(member.find("bndbox").find("ymax").text)
        boxes.append([xmin, ymin, xmax, ymax])

    return boxes, tree


# Define the augmentation pipeline
transform = A.Compose(
    [
        A.Resize(height=350, width=350),
    ],
    bbox_params=A.BboxParams(format="pascal_voc", label_fields=["class_labels"]),
)


# Function to process a single image and its XML annotation
def process_image(image_path, xml_path):
    # Load the image
    image = cv2.imread(image_path)

    # Parse the XML file to get bounding boxes
    boxes, tree = parse_voc_xml(xml_path)
    class_labels = [0] * len(boxes)  # Replace with actual class labels if available

    # Apply the augmentation
    transformed = transform(image=image, bboxes=boxes, class_labels=class_labels)

    transformed_image = transformed["image"]
    transformed_bboxes = transformed["bboxes"]

    # Save the augmented image and create a new XML file with the updated bounding boxes
    augmented_image_path = image_path.replace("images", "augmented_images")
    augmented_xml_path = xml_path.replace("images", "augmented_images")

    os.makedirs(os.path.dirname(augmented_image_path), exist_ok=True)

    cv2.imwrite(augmented_image_path, transformed_image)

    create_augmented_xml(augmented_xml_path, transformed_bboxes, tree)


# Function to create a new XML file with updated bounding boxes
def create_augmented_xml(xml_path, bboxes, original_tree):
    root = original_tree.getroot()

    for i, member in enumerate(root.findall("object")):
        bbox = member.find("bndbox")
        bbox.find("xmin").text = str(bboxes[i][0])
        bbox.find("ymin").text = str(bboxes[i][1])
        bbox.find("xmax").text = str(bboxes[i][2])
        bbox.find("ymax").text = str(bboxes[i][3])

    original_tree.write(xml_path)


# Traverse directories and process images
base_dir = "Tensorflow/workspace/images/collectedimages"
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".jpg"):
            image_path = os.path.join(root, file)
            xml_path = image_path.replace(".jpg", ".xml")

            if os.path.exists(xml_path):
                process_image(image_path, xml_path)